In [3]:
from matplotlib import *
import numpy as np
import scipy
from scipy.integrate import odeint, ode, complex_ode
from cmath import *
from pylab import *
##########
#constants
phi_0 = 3.29e-16

In [4]:
#class definition
class TWPA:
    #this guy contains TWPA parameters and can provide one with 
    #ODE functions for P,S,I.
    #All manipulations are to be made in the other class
    
    ##########################################
    #definitions and parameters set functions#
    ##########################################
    C = 4.5e-14
    CJ = 3.29e-14
    a = 10e-6
    N = 1e3
    IJ = 3.45e-6
    L = 1e-9
    wp = 6.28e9 * 6.
    ws = 6.28e9 * 7.
    wi = 1e0
    
    grid = linspace(1.,2.,10)
    gamma = 1.
    
    #initial conditions
    #yp0 = 1e-3
    #ys0 = 1e-2
    #yi0 = 1e-3
    y0 = [1e1,1e1,1e1,0.,0.,0.]  #initial condition, yp,ys,yi,thetaP,...
    y = [1e1,1e1,1e1,0.,0.,0.]    #current state (x)
    #
    
    kp = 1e1
    ks = 1e1
    ki = 1e1
    
    
    deltaKl = 1e1
    
    #Stuff
    Qp = 1e1
    Qs = 1e1
    Oi = 1e1
    x1 = 1e1
    x2 = 2e1
    Const = 1e1
    P = 1e1
    beta = 1e1
    
    #####################################
    #For phase matching
    ResNumber = 1.
    
    Cres = 6e-12
    Lres = 120e-12
    Cg = 20e-15
    Z = 20.###############should be defined carefully. Kost'l now!!!!!!
    S21Abs  = 1e-2
    S21Phase = 1e-2

    def S(self,freq):
        R = -1j /freq/self.Cg * (1 - freq**2 * self.Lres * (self.Cres + self.Cg )) / (1 - freq**2 * self.Lres * (self.Cres) )
        S21 = 1 / (1 + self.Z / 2 / R)
        #S11 =  - 1/(1 + 2*R/self.Z)
        #S0 = np.array([[S11, S21],[S21, S11]])
        return S21

    
    def SetPhaseMatching(self, n, C_res, L_res, C_g):######Call it after Initiation!!!!
        self.ResNumber = n
        self.Cres = C_res
        self.Lres = L_res
        self.Cg = C_g
        self.Z = (self.L/(self.C + self.CJ))**0.5
        
        
    def PhaseShift(self, x1, x2, y):#Make sure that GridLen >= N_cells
        if(floor(x1 / (self.a*self.ResNumber)) < floor(x2 / (self.a*self.ResNumber))):
            y[0] = y[0] * np.abs(self.S(self.wp))
            y[1] = y[1] * np.abs(self.S(self.ws))
            y[2] = y[2] * np.abs(self.S(self.wi))
            y[3] = y[3] + phase(self.S(self.wp))
            y[4] = y[4] + phase(self.S(self.ws))
            y[5] = y[5] + phase(self.S(self.wi))
    ######################################
    
    def __init__(self):
        #numbers taken from 1st Siddiqi article
        self.IJ = 3.3e-6
        self.L = phi_0/self.IJ
        self.C = 4.5e-14
        self.CJ = 3.29e-14
        self.a = 10e-6
        self.wp = 6.28e9 * 6
        self.ws = 6.28e9 * 6.1
        self.wi = 2*self.wp - self.ws
        self.N = 1e3
        
        self.gridLen = 1e3
        self.grid = linspace(0,self.N * self.a, self.gridLen)
        self.step = self.grid[1] - self.grid[0]
        
        self.WaveVectors()
        self.y0 = [1e-15,1e-20,0.,0.,0.,0.]
        self.y = np.copy(self.y0)
        self.recountGamma()
        
        
        self.beta = self.gamma*((self.C*self.L)**0.5/self.a)**5
        self.Qp = 1e1
        self.Qs = 1e1
        self.Oi = 1e1
        self.x1 = 1e1
        self.x2 = 2e1
        self.Const = 1e1
        self.P = 1e1
        
    def setIJ(self, x):
        self.IJ = x
        self.L = phi_0/self.IJ#######Careful!!!!! Changed!!!
        self.WaveVectors()
        self.recountGamma()
        
    def setC(self,x):
        self.C = x
        self.WaveVectors()
        self.recountGamma()
        
    def setCJ(self,x):
        self.CJ = x
        self.WaveVectors()
        
    def setA(self,x):
        self.a = x
        self.grid = linspace(0,self.N*self.a, self.gridLen)
        self.step = self.grid[1] - self.grid[0]
        self.WaveVectors()
        self.recountGamma()
        
    def setGridLen(self, x):#На совесть пользователя. Не совать сюда что попало!!!!
        self.gridLen = x
        self.grid = linspace(0,self.N*self.a, self.gridLen)
        self.step = self.grid[1] - self.grid[0]
        
    def setN(self,x):
        self.N = x
        self.grid = linspace(0,self.N*self.a, self.gridLen)
        self.step = self.grid[1] - self.grid[0]
        
    def setWp(self, x):
        self.wp = x
        self.w_i = 2*self.wp - self.ws
        self.WaveVectors()
        
    def setWs(self, x):
        self.ws = x
        self.wi = 2*self.wp - self.ws
        self.WaveVectors()
        
    def setY0(self,l):
        if(len(l)!=6):
            print("wrong y0 array length \n")
            return(-1)
        self.y0 = l
        
    def setY(self,l):
        if(len(l)!=6):
            print("wrong Y array length \n")
            return(-1)
        self.y = l
        
    def YtoY0(self):
        self.y = np.copy(self.y0)
    
    def WaveVectorZeroth(self,w_input):
        return(w_input * (  self.C*self.L/(1 - self.CJ*self.L * w_input**2)  )**0.5 / self.a )
    
    def WaveVectors(self):
        self.kp = self.WaveVectorZeroth(self.wp)
        self.ks = self.WaveVectorZeroth(self.ws)#No minus!!!
        self.ki = self.WaveVectorZeroth(self.wi)#No minus!!!
        self.deltaKl = 2*self.kp - self.ks - self.ki
        
    def recountGamma(self):###########Changed for test
        self.gamma = self.a**4 /(16. * self.C * self.IJ**2 * self.L**3 )
    
    ###############################################
    #now I write a functions for ODE              #
    ###############################################
    
    def FuncPump(self,q,x):#Returns [amplitude, phase]. q = [ap,as,ai, theta_p,..,..]
        ####################
        #Changed!!!
        a =  self.gamma * self.kp / self.wp**2 * 2 * self.kp*self.ks *self.ki *(self.ks + self.ki - self.kp) *q[0]*q[1]*q[2]*np.sin(2*q[3] - q[4] - q[5] + self.deltaKl * x)
        theta = self.gamma * self.kp / self.wp**2 *(2 * self.kp*self.ks *self.ki * (self.ks + self.ki - self.kp) * q[1]*q[2]*np.cos(2*q[3] - q[4] - q[5] + self.deltaKl * x)  + self.kp**2 * (self.kp**2 * q[0]**2 + 2*self.ks**2*q[1]**2 + 2*self.ki**2 * q[2]**2))
        return([a,theta])

    
    def FuncSignal(self,q,x):
        a = - self.gamma * self.ks / self.ws**2  * self.kp**2 *self.ki * (2*self.kp - self.ki) * q[0]**2 * q[2] * np.sin(2*q[3] - q[4] - q[5] + self.deltaKl * x)
        theta = self.gamma * self.ks / self.ws**2 *( self.ks**2 * (2*self.kp**2 * q[0]**2 + self.ks**2 * q[1]**2 + 2*self.ki**2 * q[2]**2) +  self.kp**2 *self.ki * (2*self.kp - self.ki) * q[0]**2 * q[2]/ q[1] *np.cos(2*q[3] - q[4] - q[5] + self.deltaKl * x))
        return([a,theta]) 
    
    
    def FuncIdler(self,q,x):
        a = - self.gamma * self.ki / self.wi**2 *  self.kp**2 *self.ks *(2*self.kp - self.ks) * q[0]**2 * q[1]*np.sin(2*q[3] - q[4] - q[5] + self.deltaKl * x)
        theta = self.gamma * self.ki / self.wi**2 *( self.ki**2 * (2*self.kp**2 * q[0]**2 + 2*self.ks**2 * q[1]**2 + self.ki**2 * q[2]**2) +  self.kp**2 *self.ks * (2*self.kp - self.ks) * q[0]**2 * q[1]/ q[2] *np.cos(2*q[3] - q[4] - q[5] + self.deltaKl * x))
        return([a,theta]) 

    
    def Function(self,x,q):#returns derivative of array. array of (a_p, a_s, a_i, thetaP, thetaS, thetaI) is on onput
        a_p,theta_p = self.FuncPump(q,x)
        a_s, theta_s = self.FuncSignal(q,x)
        a_i, theta_i = self.FuncIdler(q,x)
        b = np.array([a_p, a_s, a_i,theta_p,theta_s,theta_i])
        return(b)  

In [11]:
class TWPAmanipulator():
    #this guy can plot all graphs.
    ampl = TWPA()
    r = ode(ampl.Function).set_integrator('dop853')
    w = linspace(0.,1.,1e1)
    wStart = 0.
    wStop = 1.
    wLen = 3e1
    
    def __init__(self):
        self.wStart = 4*6.28e9
        self.wStop = 10*6.28e9
        self.wLen = 1e2
        self.setWgrid()
    
    def setWgrid(self):
        self.w = linspace(self.wStart,self.wStop,self.wLen)
    
    def setWstart(self, x): #takes FREQUENCY - in GHz
        self.wStart = x*6.28e9
        self.setWgrid()
        
    def setWstop(self, x):
        self.wStop = x*6.28e9
        self.setWgrid()
        
    def setWlen(self, x):
        self.wLen = x
        self.setWgrid()
    
    def GainFromFreq(self): #No information about Gain(N)
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        t = []
        dt = self.ampl.step
        gain = []
        idler = []
        difference = []
        theta = []
        for x in linspace(self.wStart,self.wStop,self.wLen):
            self.ampl.setWs(x)
            self.r.set_initial_value(self.ampl.y0, 0)
            while self.r.successful() and self.r.t <= T:
                self.r.integrate(self.r.t + dt)
                self.ampl.PhaseShift(self.r.t, self.r.t+ dt, self.r.y)
                #t = np.append(t,r.t)
            #gain.append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
            #idler.append(20*np.log10(self.r.y[2]/self.ampl.y0[1]))
            
            #gain.append((self.r.y[1]/self.ampl.y0[1]))
            gain.append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
            
            #theta.append(2*self.r.y[3] - self.r.y[4] - self.r.y[5] - self.ampl.deltaKl*T)
            #idler.append(20*np.log10(self.r.y[2]/self.ampl.y0[1]))
            #difference.append((self.r.y[1]/self.ampl.y0[1])**2-(self.r.y[2]/self.ampl.y0[1])**2)
            
        plot(self.w/6.28e9,gain,label="signal")
        #plot(self.w/6.28e9,theta,label="phase")
        #plot((2*self.ampl.wp-self.w)/6.28e9,idler,label = "idler")
        #plot(self.w/6.28e9,difference)
        xlabel('frequency, GHz')
        ylabel('gain, dB')
        show()
        return(gain)
    
    
    def PhaseFromN(self): #No information about Gain(N)
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        t = []
        dt = self.ampl.step
        Pump = [] #=2ThetaP  - Thetas - Thetai
        Diff = []
        Idler = []
        self.r.set_initial_value(self.ampl.y0, 0)
        while self.r.successful() and self.r.t <= T:
            self.r.integrate(self.r.t + dt)
            #Resonators.append(self.r.y[3])
            #self.ampl.PhaseShift(self.r.t, self.r.t+ dt, self.r.y)
            t = np.append(t,self.r.t)
            #gain.append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
            #idler.append(20*np.log10(self.r.y[2]/self.ampl.y0[1]))
            Pump.append(2*self.r.y[3]/pi)
            #Resonators[len(Resonators) - 1] = self.r.y[3] -  Resonators[len(Resonators) - 1]
            Idler.append((self.r.y[5]+self.r.y[4])/pi)
            Diff.append(  (2*self.r.y[3] - self.r.y[5] - self.r.y[4] + self.ampl.deltaKl*self.r.t) / pi  )
            
        plot(t/self.ampl.a,Pump,label="2*Pump")
        #plot(self.ampl.grid/self.ampl.a,Resonators,label="Signal")
        plot(t/self.ampl.a,Idler,label="Igler + Signal")
        plot(t/self.ampl.a,Diff,label="Difference")
        #plot((2*self.ampl.wp-self.w)/6.28e9,idler,label = "idler")
        #plot(self.w/6.28e9,difference)
        xlabel('JJ number')
        ylabel('Phase, pi')
        legend()
        show()
        return(Pump)
###########################
    def PhaseFromFreqAndN(self):
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        t = []
        dt = self.ampl.step
        phase = []
        k = 0
        for x in linspace(self.wStart,self.wStop,self.wLen):
            self.ampl.setWs(x)
            self.r.set_initial_value(self.ampl.y0, 0)
            phase.append([])
            while self.r.successful() and (self.r.t <= T):
                self.r.integrate(self.r.t + dt)
                #self.ampl.PhaseShift(self.r.t, self.r.t+ dt, self.r.y)
                
                phase[k].append((2*self.r.y[3] - self.r.y[4] - self.r.y[5])/pi)
                #phase[k].append((self.r.y[4] + self.r.y[5] )/pi)
                t = np.append(t,self.r.t)
            
            ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
            while (len(phase[k]) < self.ampl.gridLen ):
                phase[k].append(0.)
                
            k = k + 1     
        
        contourf(  self.ampl.grid/self.ampl.a,self.w/6.28e9, phase, 100, alpha=0.75, cmap='jet')
        colorbar()
        xlabel("Number of JJ")
        ylabel("Freq, GHz")
        show()
        return(phase)
###################  
    def GainFromFreqAndPumpSlope(self): #Delete after using
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        t = []
        dt = self.ampl.step
        slopes = linspace(0.,3e2, 3e2 )
        gain = []
        k = 0
        for x in linspace(self.wStart,self.wStop,self.wLen):
            self.ampl.setWs(x)
            gain.append([])
            for y in linspace(0.,3e2, 3e2 ):
                self.r.set_initial_value(self.ampl.y0, 0)
                while self.r.successful() and (self.r.t <= T):
                    self.r.integrate(self.r.t + dt)
                    self.r.y[3] = self.r.y[3] + y*dt
                    #self.ampl.PhaseShift(self.r.t, self.r.t+ dt, self.r.y)

                gain.append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
                    #phase[k].append((self.r.y[4] + self.r.y[5] )/pi)
            
            k = k + 1     
        
        contourf(  slopes*T/pi,self.w/6.28e9, gain, 100, alpha=0.75, cmap='jet')
        colorbar()
        xlabel("Number of JJ")
        ylabel("Freq, GHz")
        show()
        return(gain)
    
###################
    
    def GainFromPumpPower(self,Start,Stop,Length): #No information about Gain(N)
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        t = []
        array = linspace(Start,Stop,Length)
        dt = self.ampl.step
        gain = []
        buf = []
        for x in linspace(Start,Stop,Length):
            buf = np.copy(self.ampl.y0)
            buf[0] = x
            self.ampl.setY0(buf)
            self.r.set_initial_value(self.ampl.y0, 0)
            while self.r.successful() and self.r.t <= T:
                self.r.integrate(self.r.t + dt)
            gain.append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
        plot(array / 50 * self.ampl.wp /self.ampl.IJ , gain)
        xlabel('I/IJ')
        ylabel('Signal gain, dB')
        show()
        return(gain)
    
   

    def GainFromFreqAndN(self):
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        t = []
        dt = self.ampl.step
        gain = []
        band = []
        k = 0
        for x in linspace(self.wStart,self.wStop,self.wLen):
            self.ampl.setWs(x)
            self.r.set_initial_value(self.ampl.y0, 0)
            gain.append([])
            band.append([])
            while self.r.successful() and (self.r.t <= T):
                self.r.integrate(self.r.t + dt)
                self.ampl.PhaseShift(self.r.t, self.r.t+ dt, self.r.y)
                
                gain[k].append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))####!!!!Idler
                t = np.append(t,self.r.t)
            
            ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
            while (len(gain[k]) < self.ampl.gridLen ):
                gain[k].append(0.)
                
            k = k + 1     
        
        contourf(  self.ampl.grid/self.ampl.a,self.w/6.28e9, gain, 100, alpha=0.75, cmap='copper', vmin = 0.)
        colorbar()
        xlabel("Number of JJ")
        ylabel("Freq, GHz")
        show()
        print ndim(gain)
        return(gain)
    
    
    def GainFromNAndIJ(self,Istart,Istop,Ilength):
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        IJlist = linspace(Istart,Istop,Ilength)
        dt = self.ampl.step
        gain = []
        k = 0
        for x in linspace(Istart,Istop,Ilength):
            self.ampl.setIJ(x)
            self.r.set_initial_value(self.ampl.y0, 0)
            gain.append([])
            while self.r.successful() and (self.r.t <= T):
                self.r.integrate(self.r.t + dt)
                #print "hello"
                gain[k].append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
            
            ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
            while (len(gain[k]) < self.ampl.gridLen ):
                gain[k].append(0.)
                
            k = k + 1
            
        contourf(  self.ampl.grid/self.ampl.a,IJlist/1e-6, gain, 100, alpha=0.75, cmap='copper')
        colorbar()
        xlabel("Number of JJ")
        ylabel("IJ, mkA")
        show()
        return(gain)
    
    def GainFromNAndC(self,Istart,Istop,Ilength):
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        IJlist = linspace(Istart,Istop,Ilength)
        dt = self.ampl.step
        gain = []
        k = 0
        for x in linspace(Istart,Istop,Ilength):
            self.ampl.setC(x)
            self.r.set_initial_value(self.ampl.y0, 0)
            gain.append([])
            while self.r.successful() and (self.r.t <= T):
                self.r.integrate(self.r.t + dt)
                #print "hello"
                gain[k].append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
            
            ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
            while (len(gain[k]) < self.ampl.gridLen ):
                gain[k].append(0.)
                
            k = k + 1
            
        contourf(  self.ampl.grid/self.ampl.a,IJlist/1e-15, gain, 100, alpha=0.75, cmap='copper')
        colorbar()
        xlabel("Number of JJ")
        ylabel("C, fF")
        show()
        return(gain)
    
    
    def GainFromFreqAndIJ(self,Istart,Istop,Ilength):
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        IJlist = linspace(Istart,Istop,Ilength)
        dt = self.ampl.step
        gain = []
        k = 0
        for x in linspace(self.wStart,self.wStop,self.wLen):
            gain.append([])
            for i in linspace(Istart,Istop,Ilength):
            
                self.ampl.setWs(x)
                self.ampl.setIJ(i)
                self.r.set_initial_value(self.ampl.y0, 0)
                
                while self.r.successful() and (self.r.t <= T):
                    self.r.integrate(self.r.t + dt)
                    #print "hello"
                    
                gain[k].append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
                ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
                #while (len(gain[k]) < self.ampl.gridLen ):
                #    gain[k].append(0.)
            k = k + 1
            
        contourf(  IJlist*1e6 ,self.w/6.28e9, gain, 100, alpha=0.75, cmap='copper')
        colorbar()
        xlabel("IJ, mkA")
        ylabel("Freq, GHz")
        show()
        print ndim(gain)
        return(gain)
    ################################
    def GainFromFreqAndPhaseShift(self,Istart,Istop,Ilength):
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        IJlist = linspace(Istart,Istop,Ilength)
        dt = self.ampl.step
        gain = []
        k = 0
        for x in linspace(self.wStart,self.wStop,self.wLen):
            gain.append([])
            for i in linspace(Istart,Istop,Ilength):
            
                self.ampl.setWs(x)
                self.ampl.SetPhaseMatching(self.ampl.ResNumber, i)
                self.r.set_initial_value(self.ampl.y0, 0)
                
                while self.r.successful() and (self.r.t <= T):
                    self.r.integrate(self.r.t + dt)
                    self.r.y[3] = self.r.y[3] + self.ampl.PhaseShift(self.r.t, self.r.t+ dt)
                    #print "hello"
                    
                gain[k].append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
                ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
                #while (len(gain[k]) < self.ampl.gridLen ):
                #    gain[k].append(0.)
            k = k + 1
            
        contourf(  IJlist/pi ,self.w/6.28e9, gain, 100, alpha=0.75, cmap='copper')
        colorbar()
        xlabel("PhaseShift, rad")
        ylabel("Freq, GHz")
        show()
        print ndim(gain)
        return(gain)
    
    
    def GainFromPhaseAndN(self):
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        t = []
        dt = self.ampl.step
        gain = []
        band = []
        phase = linspace(-pi/2,pi/2,2e2)
        k = 0
        self.ampl.setWs(self.ampl.wp)
        
        for x in linspace(-4*pi/2,4*pi/2,2e2):
            
            self.ampl.SetPhaseMatching(self.ampl.ResNumber, x)
            self.r.set_initial_value(self.ampl.y0, 0)
            gain.append([])
            band.append([])
            while self.r.successful() and (self.r.t <= T):
                self.r.integrate(self.r.t + dt)
                self.r.y[3] = self.r.y[3] + self.ampl.PhaseShift(self.r.t, self.r.t+ dt)
                #print "hello"
                gain[k].append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
                t = np.append(t,self.r.t)
            
            ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
            while (len(gain[k]) < self.ampl.gridLen ):
                gain[k].append(0.)
                
            k = k + 1
        buf = 0.
        buf2 = 0.
        print len(gain[0])
        gainForBand = []
        gainForBand = np.copy(gain)
        gainForBand = transpose(gainForBand)
        contourf(  self.ampl.grid/self.ampl.a,phase/pi, gain, 100, alpha=0.75, cmap='copper')
        colorbar()
        xlabel("Number of JJ")
        ylabel("Freq, GHz")
        show()
        print ndim(gain)
        return(gain)
    ############################################################################
    def GainFromFreqAndY0P(self,Ystart,Ystop,Ylength):
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        Y0list = linspace(Ystart,Ystop,Ylength)
        dt = self.ampl.step
        gain = []
        k = 0
        for x in linspace(self.wStart,self.wStop,self.wLen):
            gain.append([])
            self.ampl.setWs(x)
            for i in linspace(Ystart,Ystop,Ylength):
                buff = np.copy(self.ampl.y0)
                buff[0] = i
                self.ampl.setY0(buff)
                self.r.set_initial_value(self.ampl.y0, 0)
                
                while self.r.successful() and (self.r.t <= T):
                    self.r.integrate(self.r.t + dt)
                    #print "hello"
                    
                gain[k].append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
                ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
                #while (len(gain[k]) < self.ampl.gridLen ):
                #    gain[k].append(0.)
            k = k + 1
            
        contourf(  Y0list ,self.w/6.28e9, gain, 100, alpha=0.75, cmap='copper')
        colorbar()
        xlabel("IJ, mkA")
        ylabel("Freq, GHz")
        show()
        print ndim(gain)
        return(gain)
    
    
    ########PhaseMatchingStuff
    def GainFromPumpFreqAndN(self):
        T = self.ampl.grid[len(self.ampl.grid) - 1]
        t = []
        dt = self.ampl.step
        gain = []
        k = 0
        for x in linspace(self.wStart,self.wStop,self.wLen):
            self.ampl.setWp(x)
            self.ampl.setWs(1.1*self.ampl.wp)
            self.r.set_initial_value(self.ampl.y0, 0)
            gain.append([])
            while self.r.successful() and (self.r.t <= T):
                self.r.integrate(self.r.t + dt)
                self.ampl.PhaseShift(self.r.t, self.r.t+ dt, self.r.y)
                #print "hello"
                gain[k].append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
                t = np.append(t,self.r.t)
            
            ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
            while (len(gain[k]) < self.ampl.gridLen ):
                gain[k].append(0.)
                
            k = k + 1
            
        contourf(  self.ampl.grid/self.ampl.a,self.w/6.28e9, gain, 100, alpha=0.75, cmap='copper')
        colorbar()
        xlabel("Number of JJ")
        ylabel("Freq, GHz")
        show()
        print ndim(gain)
        return(gain)

In [10]:
b = TWPAmanipulator()
b.setWstart(2.)
b.setWstop(10.)
b.setWlen(1e2)
b.ampl.setGridLen(3e2)
b.ampl.setN(3e2)
b.ampl.setA(50e-6)
b.ampl.setC(600e-15)
b.ampl.setCJ(200e-15)
b.ampl.setIJ(3.45e-6)
b.ampl.setWp(5.9204*6.28e9)
#b.ampl.setWp(5.925*6.28e9)

b.ampl.SetPhaseMatching(5., 6e-12, 120e-12,30e-15)

print 'Impedance = ', (2*b.ampl.L/(b.ampl.C + b.ampl.CJ))**0.5, 'Ohms'
print 'Cutoff Freq = ', (b.ampl.C * 2* b.ampl.L)**-0.5/6.28e9, 'GHz'
print 'Plazma Freq = ', (b.ampl.CJ * b.ampl.L)**-0.5/6.28e9, 'GHz'
#b.ampl.setWp(7*6.28e9)
#b.ampl.setC(30e-14)
#b.initiate()
b.ampl.setY0([0.6*b.ampl.IJ/b.ampl.wp*22.,1e-20,1e-23,0.,0.,0.]) #50 is impedance
#b.ampl.setY0([0.6*b.ampl.IJ/b.ampl.wp*22.,1e-20,1e-23,0.,0.,0.])
#b.ampl.setY0([0.6*b.ampl.IJ/b.ampl.wp*22.,1e-20,1e-23,0.,0.,0.])
print 'signal\pump',b.ampl.y0[1]/b.ampl.y0[0]
#b.setWstart(6.28e9*1)
#b.setWstop(6.28e9*11)




g = []
#b.ampl.setC(2*b.ampl.C)
#g = b.GainFromFreqAndPumpSlope()

#g = b.GainFromFreq()



#g = b.GainFromPumpFreqAndN()
#b.ampl.setWs(4*6.28e9)
#g = b.PhaseFromN()
#g = b.PhaseFromFreqAndN()

g = b.GainFromFreqAndN()
#g = b.GainFromFreqAndPhaseShift(-pi,pi,1e2)
#g = b.GainFromPhaseAndN()

#g = b.GainFromPumpPower(0.,b.ampl.IJ/b.ampl.wp*50.,200)
#g = b.GainFromNAndC(16e-14,30e-14,1e2)
#g[0].append(0)
#print len(g[0]), len(g[1])

Impedance =  15.4403949788 Ohms
Cutoff Freq =  14.8854281847 GHz
Plazma Freq =  36.4617036555 GHz
signal\pump 8.16427580149e-06
2


In [5]:
b = TWPAmanipulator()
b.ampl.setGridLen(5e2)
b.ampl.setN(3e2)
b.ampl.setC(45e-15)
b.ampl.setCJ(55e-15)
b.ampl.setIJ(4.6e-6)
b.ampl.setWp(5.85*6.28e9)
b.ampl.SetPhaseMatching(500., 6e-12, 120e-12,20e-15)
b.setWlen(2e4)
#
b.ampl.SetPhaseMatching(3., 6e-12, 120e-12,20e-15)
print (1/b.ampl.Lres/b.ampl.Cres)**0.5/6.28e9
T = []
Ph = []
b.setWstart(3.)
b.setWstop(8.)
Fr = linspace(b.wStart, b.wStop, b.wLen)
for k in linspace(b.wStart, b.wStop, b.wLen):
    b.ampl.setWp(k)
    T.append(b.ampl.S21Abs)
    Ph.append(b.ampl.S21Phase)
    
plot(Fr/6.28e9, T)
plot(Fr/6.28e9,Ph)
show()

5.93436299761


In [26]:
#################
#Here i picture S21 for resonator
#################

Cg = 1e-12
C = 6e-12
L = 120e-12
Z = 27.
Freq = linspace(6.28e9*3.8, 8.28e9*6,1e5)

def S(freq):
    R = 1 /(1j * freq * Cg) + 1/( 1j * freq * C + 1 / (1j * freq * L )  )  
    #R = 1j * freq * L + 1 / (1j * freq * C)
    S21 =  1/ (1 + Z / 2 / R)
    #S21 =  1/ (1 + R / 2 / Z)
    S11 =  -1/(1 + 2*R/Z)
    S0 = np.array([[S11, S21],[S21, S11]])
    return S0

N = 1.


Phase = []
Final = []
S21Re = []
S21Im = []
Sdif = []
Fun = []
dB = []
for i in range(len(Freq)):
    Final.append(S(Freq[i]))
    S21Re.append(Final[i][0][1].real)
    S21Im.append(Final[i][0][1].imag)
    dB.append(20 * np.log10(np.abs(Final[i][1][0])))
    Phase.append(phase(Final[i][1][0])/pi)
    #buf = -(Final[i][0][0]**2 -  Final[i][0][1]**2  + Final[i][0][0])/Final[i][0][1]/2.
    #Sdif.append( phase(buf))
    #Fun.append(np.abs(Final[i][0][0])**2 + np.abs(Final[i][0][1].conjugate())**2)

#plot(Freq/6.28e9, S21Re, 'b')
#plot(Freq/6.28e9, S21Im, 'g')
plot(Freq/6.28e9, dB, 'b',label = 'S21, dB')

#plot(Freq/6.28e9, Fun, 'g')
#plot(Freq/6.28e9, -1 /Freq/Cg * (1 - Freq**2 * L * (C + Cg )) / (1 - Freq**2 * L * (C) ), 'g')
plot(Freq/6.28e9, Phase, 'r', label = 'arg(S21)/pi')
xlabel('Freq, GHz')
legend()
show()
#print 1/(L*C)**0.5/6.28e9
#print polar(S21)/3.141569=

In [7]:
b.ampl.setGridLen(2e3)
plot(b.ampl.grid/b.ampl.a, floor(b.ampl.grid / (b.ampl.a*b.ampl.ResNumber)) )
show()

NameError: name 'b' is not defined

In [ ]:
print 0.91*b.ampl.IJ/b.ampl.wp*50.

In [ ]:
x = linspace(1.,1e2,1e4)
y = 1/2/1j * np.arctan(np.exp(1j*x))
plot(x, y.imag)
show()

In [ ]:
print (20e-15/(8e-12*4/1.7e-7))**0.5

In [15]:
print b.ampl.L*3e9

0.214565217391


In [ ]:
class DataFor2dPlot():
    x = []
    array = []
    legend = ''
    xLabel = ''
    yLabel = ''
    
    def plotGraph(self):
        plot(x,array, label = legend)
        legend()
        xLabel = self.xLabel
        yLabel = self.yLabel
        
class DataFor3dPlot():
    x = []
    y = []
    array = []
    legend = ''
    xLabel = ''
    yLabel = ''
    title = ''
    cBarLabel = ''
    
    def plotGraph(self):
        contourf(  freq,NumberList, dbAmplitude, 100, alpha=0.75, cmap='Spectral')
        cbar = colorbar()
        cbar.set_label(cBarLabel, rotation=90)
        title(self.title)
        ylabel(self.yLabel)
        xlabel(self.xLabel)
    